# Import des libraries

In [ ]:
!pip install fsspec==2025.3.2


In [ ]:
!pip install -q transformers datasets torchvision pycocotools


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 100.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 89.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 52.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 100.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

# Importation

In [ ]:
from torch.utils.data import DataLoader, Subset
import torchvision.transforms as T
from pycocotools.coco import COCO
from torchvision.datasets import CocoDetection

import numpy as np
import random

import torch
from transformers import AutoImageProcessor, DeformableDetrForObjectDetection, Trainer, TrainingArguments
from datasets import load_dataset
from torchvision import transforms

from tqdm import tqdm
import os

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("CUDA disponible :", torch.cuda.is_available())
if torch.cuda.is_available():
    print("Nom du GPU :", torch.cuda.get_device_name(0))
else:
    print("Aucun GPU détecté")


CUDA disponible : True
Nom du GPU : Tesla T4


# Préparation du dataset

In [ ]:
!unzip -q /content/data.zip -d /content/custom_data_raw

```
data/
├── images/
│   ├── train/
│   │   ├── 000000000001.jpg
│   │   ├── 000000000002.jpg
│   │   ├── ...
│   └── val/
│       ├── 000000000001.jpg
│       ├── 000000000002.jpg
│       ├── ...
├── annotations/
│   ├── instances_train2017.json
│   ├── instances_val2017.json


In [ ]:
import os
import shutil
from sklearn.model_selection import train_test_split

# Répertoire d'origine pour les images et annotations
raw_data_path_images = "/content/custom_data_raw/images"  # Images sont dans ce dossier
raw_data_path_labels = "/content/custom_data_raw/obj_train_data"  # Annotations dans ce dossier

# Obtenir toutes les images (jpg/png) présentes dans le dossier "images"
images = [f for f in os.listdir(raw_data_path_images) if f.endswith(('.jpg', '.png'))]

# Split train/val (80% train, 20% val)
train_imgs, val_imgs = train_test_split(images, test_size=0.2, random_state=42)

# Création des dossiers cible pour images et labels
base_path = "/content/custom_data"


os.makedirs(base_path + "/images/train", exist_ok=True)
os.makedirs(base_path + "/images/val", exist_ok=True)
os.makedirs(base_path + "/labels/train", exist_ok=True)
os.makedirs(base_path + "/labels/val", exist_ok=True)

def move_data(image_list, split):
    for img_name in image_list:
        # Déplacer l'image vers le bon dossier
        src_img = os.path.join(raw_data_path_images, img_name)
        dst_img = os.path.join(base_path, f"images/{split}", img_name)
        shutil.copy(src_img, dst_img)

        # Vérifier et déplacer l'annotation .txt correspondante
        txt_name = img_name.rsplit('.', 1)[0] + '.txt'
        txt_src = os.path.join(raw_data_path_labels, txt_name)
        txt_dst = os.path.join(base_path, f"labels/{split}", txt_name)

        if os.path.exists(txt_src):
            shutil.copy(txt_src, txt_dst)
        else:
            print(f"Pas d'annotation pour {img_name}")

# Appliquer le déplacement aux deux splits : train et val
move_data(train_imgs, "train")
move_data(val_imgs, "val")


In [ ]:
import os
import cv2
import json
from tqdm import tqdm

tile_size = 320
overlap = 0
visualize = True  # Active la visualisation des tiles avec boxes
splits = ["train", "val"]

input_root = "/content/custom_data"
output_root = "/content/split_custom_data"
visu_dir = "/content/visu_tiles"

categories = []

def yolo_to_bbox(x_center, y_center, w, h, img_w, img_h):
    x1 = (x_center - w / 2) * img_w
    y1 = (y_center - h / 2) * img_h
    w *= img_w
    h *= img_h
    return x1, y1, w, h

for split in splits:
    image_id = 0
    annotation_id = 0

    input_img_dir = os.path.join(input_root, f"images/{split}")
    input_lbl_dir = os.path.join(input_root, f"labels/{split}")
    output_img_dir = os.path.join(output_root, f"{split}/images")
    output_ann_path = os.path.join(output_root, f"{split}/instances_{split}.json")
    output_visu_dir = os.path.join(visu_dir, split)
    os.makedirs(output_img_dir, exist_ok=True)
    if visualize:
        os.makedirs(output_visu_dir, exist_ok=True)

    coco_dict = {
        "images": [],
        "annotations": [],
        "categories": [],
    }

    label_set = set()

    for filename in tqdm(os.listdir(input_img_dir), desc=f"{split}"):
        if not filename.lower().endswith(('.jpg', '.png')):
            continue

        basename = os.path.splitext(filename)[0]
        img_path = os.path.join(input_img_dir, filename)
        label_path = os.path.join(input_lbl_dir, f"{basename}.txt")

        img = cv2.imread(img_path)
        if img is None:
            continue
        img_h, img_w = img.shape[:2]

        annots = []
        if os.path.exists(label_path):
            with open(label_path, "r") as f:
                for line in f.readlines():
                    parts = line.strip().split()
                    if len(parts) == 5:
                        cls, xc, yc, w, h = map(float, parts)
                        annots.append((int(cls), *yolo_to_bbox(xc, yc, w, h, img_w, img_h)))
                        label_set.add(int(cls))

        step = tile_size - overlap
        tile_id = 0

        for y in range(0, img_h, step):
            for x in range(0, img_w, step):
                tile = img[y:y+tile_size, x:x+tile_size]
                th, tw = tile.shape[:2]
                if th < tile_size or tw < tile_size:
                    continue

                tile_fname = f"{basename}_{tile_id}.jpg"
                tile_path = os.path.join(output_img_dir, tile_fname)
                cv2.imwrite(tile_path, tile)

                tile_annot = tile.copy()

                coco_dict["images"].append({
                    "id": image_id,
                    "width": tile_size,
                    "height": tile_size,
                    "file_name": tile_fname,
                })

                for cls, x1, y1, w, h in annots:
                    x2 = x1 + w
                    y2 = y1 + h

                    if x1 >= x + tile_size or x2 <= x or y1 >= y + tile_size or y2 <= y:
                        continue

                    box_x1 = max(0, x1 - x)
                    box_y1 = max(0, y1 - y)
                    box_x2 = min(tile_size, x2 - x)
                    box_y2 = min(tile_size, y2 - y)

                    box_w = box_x2 - box_x1
                    box_h = box_y2 - box_y1

                    if box_w < 1 or box_h < 1:
                        continue

                    coco_dict["annotations"].append({
                        "id": annotation_id,
                        "image_id": image_id,
                        "category_id": cls,
                        "bbox": [box_x1, box_y1, box_w, box_h],
                        "area": box_w * box_h,
                        "iscrowd": 0,
                    })
                    annotation_id += 1

                    if visualize:
                        cv2.rectangle(tile_annot, (int(box_x1), int(box_y1)), (int(box_x2), int(box_y2)), (0, 255, 0), 1)
                        cv2.putText(tile_annot, str(cls), (int(box_x1), int(box_y1) - 2), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255, 255, 255), 1)

                if visualize:
                    vis_tile_path = os.path.join(output_visu_dir, tile_fname)
                    cv2.imwrite(vis_tile_path, tile_annot)

                image_id += 1
                tile_id += 1

    coco_dict["categories"] = [
        {"id": cls, "name": str(cls), "supercategory": "none"}
        for cls in sorted(label_set)
    ]

    with open(output_ann_path, "w") as f:
        json.dump(coco_dict, f, indent=2)

    print(f"✅ COCO json sauvegardé dans : {output_ann_path}")


In [ ]:
!ls /content/split_custom_data/train/images | head
!ls /content/split_custom_data/train | grep json


20221108_112521_0.jpg
20221108_112521_10.jpg
20221108_112521_11.jpg
20221108_112521_12.jpg
20221108_112521_13.jpg
20221108_112521_14.jpg
20221108_112521_15.jpg
20221108_112521_16.jpg
20221108_112521_17.jpg
20221108_112521_18.jpg
instances_train.json


Les étapes précèdentes sont là afin que le dataset possède le format voulu.

# Training


In [ ]:
dataset_path = "/content/split_custom_data"
output_path = "/content/output_olives"

## Chargement dataset

In [ ]:
import json

with open('/content/split_custom_data/train/instances_train.json', 'r') as f:
    data = json.load(f)
    print(json.dumps(data, indent=4))


{
    "images": [
        {
            "id": 0,
            "width": 320,
            "height": 320,
            "file_name": "20221108_112526_0.jpg"
        },
        {
            "id": 1,
            "width": 320,
            "height": 320,
            "file_name": "20221108_112526_1.jpg"
        },
        {
            "id": 2,
            "width": 320,
            "height": 320,
            "file_name": "20221108_112526_2.jpg"
        },
        {
            "id": 3,
            "width": 320,
            "height": 320,
            "file_name": "20221108_112526_3.jpg"
        },
        {
            "id": 4,
            "width": 320,
            "height": 320,
            "file_name": "20221108_112526_4.jpg"
        },
        {
            "id": 5,
            "width": 320,
            "height": 320,
            "file_name": "20221108_112526_5.jpg"
        },
        {
            "id": 6,
            "width": 320,
            "height": 320,
            "file_name": "20221108

In [ ]:
import json

def validate_json(json_path):
    with open(json_path, 'r') as f:
        data = json.load(f)

    # Vérifie la structure des images
    for img in data.get('images', []):
        if 'id' not in img or 'file_name' not in img:
            print(f"Image invalide: {img}")

    # Vérifie la structure des annotations
    for ann in data.get('annotations', []):
        if 'id' not in ann or 'image_id' not in ann or 'category_id' not in ann or 'bbox' not in ann:
            print(f"Annotation invalide: {ann}")

    # Vérifie les catégories
    for cat in data.get('categories', []):
        if 'id' not in cat or 'name' not in cat:
            print(f"Catégorie invalide: {cat}")

    print("Validation terminée.")
    return data

# Valide ton fichier JSON
data = validate_json('/content/split_custom_data/train/instances_train.json')

# Si la validation est OK, sauvegarde le fichier corrigé
with open('/content/split_custom_data/train/instances_train_fixed.json', 'w') as f:
    json.dump(data, f, indent=4)


Validation terminée.


In [ ]:
import json
import os

def coco_to_huggingface(coco_json_path, output_json_path):
    with open(coco_json_path, 'r') as f:
        coco_data = json.load(f)

    images = coco_data['images']
    annotations = coco_data['annotations']
    categories = {cat['id']: cat['name'] for cat in coco_data['categories']}

    # Préparer un dictionnaire par image avec ses annotations
    image_data = {}
    for img in images:
        image_data[img['id']] = {
            'image': img['file_name'],
            'annotations': []
        }

    # Ajouter les annotations aux images correspondantes
    for ann in annotations:
        image_id = ann['image_id']
        if image_id in image_data:
            image_data[image_id]['annotations'].append({
                'category_id': ann['category_id'],
                'bbox': ann['bbox'],
                'area': ann['area'],
                'iscrowd': ann['iscrowd']
            })

    # Convertir en format Hugging Face
    hf_data = []
    for img_id, img_info in image_data.items():
        hf_data.append({
            'image': img_info['image'],
            'annotations': img_info['annotations']
        })

    # Sauvegarder le fichier JSON pour Hugging Face
    with open(output_json_path, 'w') as f:
        json.dump(hf_data, f, indent=4)

# Chemin d'entrée et de sortie
input_json_path = '/content/split_custom_data/train/instances_train.json'
output_json_path = '/content/split_custom_data/train/instances_train_huggingface.json'

input_json_path2 = '/content/split_custom_data/val/instances_val.json'
output_json_path2 = '/content/split_custom_data/val/instances_val_huggingface.json'


# Conversion du fichier COCO en format Hugging Face
coco_to_huggingface(input_json_path, output_json_path)

coco_to_huggingface(input_json_path2, output_json_path2)


In [ ]:
from datasets import Dataset, DatasetDict
import os

# Chemins vers les fichiers JSON transformés
train_json = os.path.join(dataset_path, "train/instances_train_huggingface.json")
val_json = os.path.join(dataset_path, "val/instances_val_huggingface.json")

# Charger le dataset à partir des fichiers JSON
train_dataset = Dataset.from_json(train_json)
val_dataset = Dataset.from_json(val_json)

# Créer un dictionnaire de datasets
dataset = DatasetDict({
    "train": train_dataset,
    "validation": val_dataset
})

# Afficher un échantillon
print(dataset["train"][22])


Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

{'image': '20221108_112526_22.jpg', 'annotations': [{'area': 260.87671024844497, 'bbox': [182.20035599999994, 311.8981120000001, 32.199495999999954, 8.101887999999917], 'category_id': 0, 'iscrowd': 0}]}


## Pré-traitement

### Batch = True

In [ ]:
train_dataset[22]


{'image': '20221108_112526_22.jpg',
 'annotations': [{'area': 260.87671024844497,
   'bbox': [182.20035599999994,
    311.8981120000001,
    32.199495999999954,
    8.101887999999917],
   'category_id': 0,
   'iscrowd': 0}]}

In [ ]:
import gc
from PIL import Image
from transformers import AutoImageProcessor
import os

# Charge le modèle d'image processor
image_processor = AutoImageProcessor.from_pretrained("facebook/detr-resnet-50")

# Chemins des répertoires des images
train_image_directory = '/content/split_custom_data/train/images/'
val_image_directory = '/content/split_custom_data/val/images/'

def process_batch(batch, dataset_type):
    image_directory = train_image_directory if dataset_type == 'train' else val_image_directory
    pixel_values = []
    labels = []

    for img_name, annotation_list in zip(batch["image"], batch["annotations"]):
        image_path = os.path.join(image_directory, img_name)

        if not os.path.exists(image_path):
            print(f"Image non trouvée : {image_path}")
            continue

        try:
            image = Image.open(image_path).convert("RGB")
            width, height = image.size

            encoding = image_processor(images=image, return_tensors="pt")
            pixel_values.append(encoding["pixel_values"][0].tolist())  # convertit le tensor en liste

            boxes = []
            class_labels = []

            for obj in annotation_list:
                x, y, w, h = obj["bbox"]
                norm_box = [
                    x / width,
                    y / height,
                    w / width,
                    h / height
                ]
                boxes.append(norm_box)
                class_labels.append(obj["category_id"])

            labels.append({
                "class_labels": class_labels,  # pas de tensor ici
                "boxes": boxes
            })

        except Exception as e:
            print(f"Erreur sur {image_path} : {e}")

    return {"pixel_values": pixel_values, "labels": labels}

def filter_images_with_boxes(example):
    return len(example["annotations"]) > 0

# 1. Filtrer les datasets en amont
train_dataset = train_dataset.filter(filter_images_with_boxes)
val_dataset = val_dataset.filter(filter_images_with_boxes)



# Appliquer la transformation aux datasets avec batched=True
train_dataset = train_dataset.map(lambda x: process_batch(x, dataset_type='train'), batched=True, batch_size=32)
val_dataset = val_dataset.map(lambda x: process_batch(x, dataset_type='val'), batched=True, batch_size=32)

# Nettoyage mémoire
gc.collect()


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/290 [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


Filter:   0%|          | 0/2160 [00:00<?, ? examples/s]

Filter:   0%|          | 0/552 [00:00<?, ? examples/s]

Map:   0%|          | 0/999 [00:00<?, ? examples/s]

Map:   0%|          | 0/265 [00:00<?, ? examples/s]

18

## Test opti

In [ ]:
import gc
from PIL import Image
from transformers import AutoImageProcessor
import os

# Charge le modèle d'image processor
image_processor = AutoImageProcessor.from_pretrained("facebook/detr-resnet-50")

# Chemins des répertoires des images
train_image_directory = '/content/split_custom_data/train/images/'
val_image_directory = '/content/split_custom_data/val/images/'

def process_batch(batch, dataset_type):
    image_directory = train_image_directory if dataset_type == 'train' else val_image_directory
    pixel_values = []
    boxes_list = []
    class_labels_list = []

    for img_name, annotation_list in zip(batch["image"], batch["annotations"]):
        image_path = os.path.join(image_directory, img_name)

        if not os.path.exists(image_path):
            print(f"Image non trouvée : {image_path}")
            continue

        try:
            image = Image.open(image_path).convert("RGB")
            width, height = image.size

            boxes = []
            class_labels = []

            for obj in annotation_list:
                x, y, w, h = obj["bbox"]
                if w <= 0 or h <= 0:
                    continue

                norm_box = [
                    x / width,
                    y / height,
                    w / width,
                    h / height
                ]
                boxes.append(norm_box)
                class_labels.append(obj["category_id"])

            if len(boxes) == 0:
                continue

            encoding = image_processor(images=image, return_tensors="pt")
            pixel_values.append(encoding["pixel_values"][0].numpy())
            boxes_list.append(torch.tensor(boxes) * torch.tensor([width, height, width, height]))
            class_labels_list.append(torch.tensor(class_labels))

        except Exception as e:
            print(f"Erreur sur {image_path} : {e}")
            continue

    # Cas : aucun élément valide dans le batch
    if len(pixel_values) == 0:
        return {
            "pixel_values": [],
            "boxes": [],
            "class_labels": []
        }

    return {
        "pixel_values": pixel_values,
        "boxes": [b.numpy() for b in boxes_list],
        "class_labels": [cl.numpy() for cl in class_labels_list]
    }


def remove_skipped(example):
    return "skip" not in example

def remove_empty(example):
    return len(example["boxes"]) > 0






train_dataset = train_dataset.map(lambda x: process_batch(x, dataset_type='train'),
                                  batched=True,
                                  batch_size=8,
                                  remove_columns=train_dataset.column_names)

val_dataset = val_dataset.map(lambda x: process_batch(x, dataset_type='val'),
                                batched=True,
                                batch_size=8,
                                remove_columns=val_dataset.column_names)

train_dataset = train_dataset.filter(remove_empty)
val_dataset = val_dataset.filter(remove_empty)



# Nettoyage mémoire
gc.collect()


## Charger le modèle

In [ ]:
NUM_CLASSES = 2

In [ ]:
from transformers import DeformableDetrForObjectDetection

model = DeformableDetrForObjectDetection.from_pretrained("SenseTime/deformable-detr",num_labels=NUM_CLASSES,ignore_mismatched_sizes=True)


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:2397: UserWarning: for conv1.weight: copying from a non-meta parameter in the checkpoint to a meta parameter in the current model, which is a no-op. (Did you mean to pass `assign=True` to assign items in the state dictionary to their corresponding key in the module instead of copying them in place?)
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:2397: UserWarning: for bn1.weight: copying from a non-meta parameter in the checkpoint to a meta parameter in the current model, which is a no-op. (Did you mean to pass `assign=True` to assign items in the state dictionary to their corresponding key in the module instead of copying them in place?)
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:2397: UserWarning: for bn1.bias: copying from a non-meta parameter in the checkpoint to a meta parameter in the current model, which is a no-op. (Did you mean to pas

In [ ]:
model.to(device)


DeformableDetrForObjectDetection(
  (model): DeformableDetrModel(
    (backbone): DeformableDetrConvModel(
      (conv_encoder): DeformableDetrConvEncoder(
        (model): FeatureListNet(
          (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
          (bn1): DeformableDetrFrozenBatchNorm2d()
          (act1): ReLU(inplace=True)
          (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
          (layer1): Sequential(
            (0): Bottleneck(
              (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
              (bn1): DeformableDetrFrozenBatchNorm2d()
              (act1): ReLU(inplace=True)
              (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
              (bn2): DeformableDetrFrozenBatchNorm2d()
              (drop_block): Identity()
              (act2): ReLU(inplace=True)
              (aa): Identity()
              (conv3):

## Config

In [ ]:
from transformers import TrainingArguments


In [ ]:
training_args = TrainingArguments(
    output_dir=output_path,
    learning_rate=5e-5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=10,
    weight_decay=0.01,
    logging_dir=os.path.join(output_path, "logs"),
    logging_steps=10,
    eval_steps=100,
    save_steps=200,
    save_total_limit=3,
    load_best_model_at_end=True,# <-- Désactivé
    save_strategy="epoch", # Aligned save strategy with evaluation strategy
    eval_strategy="epoch", # <-- Désactivé
    fp16=True,
)


## Initialisation

In [ ]:
def collate_fn(batch):
    # Reconstruit les pixel_values à partir des listes
    pixel_values = [torch.tensor(example["pixel_values"]) for example in batch]
    pixel_values = torch.stack(pixel_values)

    # Préparer les labels au bon format pour DETR
    labels = []
    for example in batch:
        boxes = torch.tensor(example["labels"]["boxes"], dtype=torch.float32)
        class_labels = torch.tensor(example["labels"]["class_labels"], dtype=torch.int64)

        labels.append({
            "class_labels": class_labels,
            "boxes": boxes
        })

    return {
        "pixel_values": pixel_values,
        "labels": labels  # DETR attend une liste de dicos
    }


In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=collate_fn,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

In [ ]:
import os
os.environ["WANDB_DISABLED"] = "true"
os.environ["WANDB_MODE"] = "disabled"



In [ ]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Epoch,Training Loss,Validation Loss
1,1.578600,1.494264
2,1.199400,1.203897
3,0.995400,1.092480
4,1.089800,1.041959
5,0.933500,1.013113


# Sauvegarde

In [ ]:
model.save_pretrained(os.path.join(output_path, "final_model"))
image_processor.save_pretrained(os.path.join(output_path, "final_processor"))

print("Entraînement terminé et modèle sauvegardé.")

In [ ]:
!mkdir -p /content/DETR
!mv /content/output_olives/final_model /content/DETR/
!mv /content/output_olives/final_processor /content/DETR/


In [ ]:
!zip -r /content/DETR.zip /content/DETR